In [51]:

!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreaceated sklearn

Task 1

In [52]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./online_shoppers_intention.csv"))
print(frame)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [53]:
#visualisation

Verhaaltje van ~1min

Task 2

In [56]:
#Prepositioning op basis van algoritmes in task 3
# Preprocess using dropna AND change all string types to an integer
def Month_to_num(x):
    months = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
        "May": 5, "June": 6, "Jul": 7, "Aug": 8,
        "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12, "Returning_Visitor": 1, "Old_Visitor": 2, "New_Visitor": 3, "Other": 4}
    if x not in months:
        raise ValueError(f"Invalid month string: {x}")
    return months[x]

addlist = []
preprocessed = (frame.copy().dropna())
for i,j in enumerate(preprocessed['Month']):
    addlist.append(Month_to_num(j))
preprocessed['Month'] = addlist

addlist = []
for i,j in enumerate(preprocessed['VisitorType']):
    addlist.append(Month_to_num(j))
preprocessed['VisitorType'] = addlist

for i in preprocessed.columns:
    preprocessed[i] = pd.to_numeric(preprocessed[i], errors='raise')

print(preprocessed)



       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

Task 3

In [55]:
# affinity propagation clustering - aan de hand van documentatie - NU HEEL SLOOM
from sklearn.cluster import AffinityPropagation
Data = preprocessed.copy()
clustering = AffinityPropagation(random_state=5).fit(Data)
AffinityPropagation(random_state=24)

labels = clustering.labels_
cluster_centers = clustering.cluster_centers_
n_clusters = len(cluster_centers)

print("Number of clusters:", n_clusters)
print("Cluster centers:\n", cluster_centers)


# DBSCAN clustering

# Birch clustering

KeyboardInterrupt: 

Task 4

In [ ]:
# manual Silhouette score

# Davies Buildin Score

# calinski-harabasz index

Task 5 ALL MANUAL


In [ ]:
# Euclidian distance

# Manhattan distance

# Cosine similarity

# Distances effect on dbscan